# Time to Train my first LLM
---



## Setup

In [1]:
# install unsloth and pandas
#!pip install unsloth
#!pip install pandas

In [ ]:
# checking if I can use my gpu
import torch
torch.cuda.is_available()

True

In [3]:
# import the libs that our teacher told us to
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.7.0+cu118)
    Python  3.12.10 (you have 3.12.3)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# set the max number of tokens that can be passed in per chunk
max_seq_length = 2048

# let unsloth select our data type for us
dtype = None

# enable 4-bit quantization
# less memory but slightly less accuracy its just some math stuff
load_in_4bit = True

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
# find path to my data
#!ls 'drive/MyDrive/intro to AI/Notebooks'

In [7]:
import json
import pandas as pd
from datasets import Dataset

# connect to our data path
#dataPath = 'drive/MyDrive/intro to AI/Notebooks/allMethodsCombined.json'

dataPath = '../data/allMethodsCombined.json'

# load and convert to DataFrame
with open(dataPath, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)  # <- make it a DataFrame

# save the DataFrame to CSV
df.to_csv('allMethodsCombined.csv', index=False)

# convert to a hf dataset
dataset = Dataset.from_pandas(df)


In [8]:
# make a 2 col dataset
df2col = df[['code', 'javadoc']]

dataset2col = Dataset.from_pandas(df)


In [9]:
df2col.head(4)

,code,javadoc
0,public Point getLoc(){return location;},/*** Gets the location.\n*\n* @return the curr...
1,public void draw(Graphics g){\ng.setColor(colo...,"/*** Draws the ball, and if non-active also dr..."
2,public boolean isDone(){\nreturn finishedInstr...,/*** Checks if the process is completed.\n* @r...
3,public void performInstruction(){\nif (!isDone...,/**Perform a single instruction of the process...


## Model Selections:
- Qwen 2.5 coder (0.5b)
- Codellama (7b)
- Wizardcoder (33b)


In [10]:
# models from hugging face
model1 = 'Qwen/Qwen2.5-Coder-0.5B-Instruct'
model2 = 'WizardLMTeam/WizardCoder-33B-V1.1'
model3 = 'codellama/CodeLlama-7b-hf'


## Fine tuning Qwen 2.5 Coder (0.5B)

### Setting up model

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # qwen coder 0.5b
    model_name= model1,
    max_seq_length=max_seq_length,
    dtype = dtype,
    # give each model 4bit quantization or however you spell it
    load_in_4bit= True
)

c:\Users\eyas1\OneDrive\Desktop\CS 422 Project\422Project\venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.4.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3080 Ti. Num GPUs = 1. Max memory: 12.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Establish How the model will learn

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.4.3 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


### Start Training (Can be changed)

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset2col,
    dataset_text_field="code",
    args=TrainingArguments(
        output_dir="output",
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=2e-4,
        report_to="none",
    ),
    dataset_num_proc=1,  # <- add this line to avoid crashing
)


Unsloth: Tokenizing ["code"]:   0%|          | 0/161 [00:00<?, ? examples/s]

In [14]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 161 | Num Epochs = 2 | Total steps = 162
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss


TrainOutput(global_step=162, training_loss=0.014899389243420259, metrics={'train_runtime': 45.6245, 'train_samples_per_second': 7.058, 'train_steps_per_second': 3.551, 'total_flos': 73639954982400.0, 'train_loss': 0.014899389243420259})

## Upload to Hugging Face

In [ ]:
from huggingface_hub import login

# Log in first (you only need to do this once)
login()

# Push model
model.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")
tokenizer.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")


README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

Saved model to https://huggingface.co/Eyas1/fine_tuned_qwen_java_docs


README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
